# Prediction of Nighttime NO2

## Background

### Names and Acronyms
1) ASDC: [Atmosperhic Science Data Center](https://asdc.larc.nasa.gov/about)
1) PGN (Pandora): [Pandonia Global Network](https://www.pandonia-global-network.org/) / [Pandora](https://pandora.gsfc.nasa.gov/About/)
    - **NOTE**: NASA's portion of the PGN is known as Pandora.  Within the scope of this notebook, Pandora and PGN may be used interchangably as this project will only use NASAs PGN site data.
1) TEMPO: [Troposoperic Emissions: Monitoring of Pollution](https://science.nasa.gov/mission/tempo/)

### Resources
1) ASDC Data Processing Tool (Version 1)
    - This notebook was published by the ASDC and provides examples of how to correctly load and use Pandora and TEMPO data.
    - https://github.com/nasa/ASDC_Data_and_User_Services/blob/main/TEMPO/additional_drafts/ASDC_Data_Processing_ML_v1.2.ipynb
1) PGN Station Map
    - A map showing the location of all PGN groundsites.
    - https://blickm.hetzner.pandonia-global-network.org/livemaps/pgn_stationsmap.png


This notebook borrows heavily from and extens the functionality of the NASA, ASDC Data and User Servicies notebook found here:

https://github.com/nasa/ASDC_Data_and_User_Services/blob/main/TEMPO/additional_drafts/ASDC_Data_Processing_ML_v1.2.ipynb

This notebook intends to test the hypothesis that a model can be built with Pandora which can predict nightitme NO<sub>2</sub> and that that model can be applied to TEMPO daytime measurments to predict NO<sub>2</sub> for any location covered by TEMPO.



## 1. Environment Setup

### Environment Setup
There are many tools available such as [poetry](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://python-poetry.org/&ved=2ahUKEwjr9aLgna6QAxX5EVkFHVsNBMUQFnoECBsQAQ&usg=AOvVaw3Jp8q7OO7XkcY8Tq4tDe30) and [uv](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://docs.astral.sh/uv/&ved=2ahUKEwiP9aXVna6QAxVyF1kFHeyTNGYQFnoECAsQAQ&usg=AOvVaw2VJVt0jrah2S9tIgdc1yRc) that simplify and speed up environment setup.  For simplicity, this guide only covers the method built into the python standard library.
1) Install [Python 3.11](https://www.python.org/downloads/) (or higher)
1) (Recomended) Create a virtual environment (learn more [here](https://docs.python.org/3/library/venv.html))
1) Install the required packages using the following command.<br>`% pip install pyproject.toml`
1) Select the newly created kernal in your notebook.
    - NOTE: this varies slightly between notebook tools, but in almost all tools you will be prompted to select a kernal upon running a cell.

### Import required modules

In [61]:
from datetime import datetime, timedelta
from pathlib import Path
import warnings

import earthaccess
import netCDF4 as nc
import numpy as np
import pandas as pd
import requests

### Data Access
In order to access data, you will need an Earthdata Login account.  If you do not have an Earthdata Login account, you can create one here:<br>
https://urs.earthdata.nasa.gov/

The earthaccess module allows you to authenticate.  Multilple login options exist for providing your credentials, you can read more on options here:<br>
https://pypi.org/project/earthaccess/<br>
By unless another option is configured, you will be prompted by your notebook to enter your credentials.

In [62]:
earthaccess.login()

Notebook Settings

In [63]:
PGN_DATA_DIR = Path('pgn-data')
PGN_DATA_DIR.mkdir(mode=0o777, parents=True, exist_ok=True)
PGN_DATA_PATH = PGN_DATA_DIR.joinpath('pgn-data.csv')
TEMPO_DATA_DIR = Path('tempo-data')
TEMPO_DATA_DIR.mkdir(mode=0o777, parents=True, exist_ok=True)
TEMPO_DATA_PATH = TEMPO_DATA_DIR.joinpath('tempo-data.csv')

## 1. Data Prepairation

### 1.1. Download Data

#### 1.1.1. Define Download Settings

In [64]:
start_date = datetime(2024, 7, 1)
end_date = datetime(2024, 7, 7)
temporal_range = [start_date, end_date]
spatial_range = []

sites = ['BronxNY', 'BuffaloNY', 'QueensNY']
sites_url = "https://data.pandonia-global-network.org"

# settings for various types of pgn files
all_pgn_formats = {
    "rnvh3p1-8.txt": {
        'no2_quality_flag_index': 52,
        'valid_quality_flags': [0, 10],
        'column_index': 61,
        'column_unc_index': 62,
    },
    "rnvm2p1-8.txt": {
        'no2_quality_flag_index': 35,
        'valid_quality_flags': [0, 1, 10, 11],
        'column_index': 38,
        'column_unc_index': 39,
    }
}

# formats to process
pgn_formats = {
    "rnvh3p1-8.txt": all_pgn_formats["rnvh3p1-8.txt"]
}

#### 1.1.1. Download Pandora data

Get sites

In [65]:
def get_page_links(url: str):
    """
    An tool for getting all PGN links from a PGN data webpage (https://data.hetzner.pandonia-global-network.org/)

    ARGS:
        url (str): The URL of the page to extract links form
    """
    things_to_remove = [
        '<span class="name">',
        '</span>',
        '/</span>'
    ]

    response = requests.get(url)
    assert response.status_code==200, f"Download failed with code {response.status_code}"
    
    # get item name lines
    names = [l.strip() for l in response.text.splitlines()]
    names = [l for l in names if l.startswith('<span class="name">')]

    # get item names from name lines
    for thing_to_remove in things_to_remove:
        names = [l.replace(thing_to_remove, '') for l in names]
        names = [l.rstrip('/') for l in names]

    return names

Build file URLs

In [66]:
# get file URLs
print("Getting File URLs")
pgn_urls: list[str] = []
for i, site in enumerate(sites):
    site_url = f"{sites_url}/{site}"
    instruments = get_page_links(site_url)
    print(f"Site {i+1} of {len(sites)}:", site)
    for j, instrument in enumerate(instruments):
        print(f"\tInstrument {j+1} of {len(instruments)}:", instrument)
        for file_suffix in pgn_formats.keys():
            file_url = f"{site_url}/{instrument}/L2/{instrument}_{site}_L2_{file_suffix}"
            # verify file exists
            if not requests.head(file_url, allow_redirects=True).ok:
                print(f"\tFile does not exist (this may not be an issue): {file_url}")
                continue
            pgn_urls.append(file_url)

print("File URLs:")
for pgn_url in pgn_urls:
    print(f"\t{pgn_url}")


Getting File URLs
Site 1 of 3: BronxNY
	Instrument 1 of 2: Pandora147s1
	Instrument 2 of 2: Pandora180s1
Site 2 of 3: BuffaloNY
	Instrument 1 of 1: Pandora206s1
Site 3 of 3: QueensNY
	Instrument 1 of 1: Pandora55s1
File URLs:
	https://data.pandonia-global-network.org/BronxNY/Pandora147s1/L2/Pandora147s1_BronxNY_L2_rnvh3p1-8.txt
	https://data.pandonia-global-network.org/BronxNY/Pandora180s1/L2/Pandora180s1_BronxNY_L2_rnvh3p1-8.txt
	https://data.pandonia-global-network.org/BuffaloNY/Pandora206s1/L2/Pandora206s1_BuffaloNY_L2_rnvh3p1-8.txt
	https://data.pandonia-global-network.org/QueensNY/Pandora55s1/L2/Pandora55s1_QueensNY_L2_rnvh3p1-8.txt


Download files (if not already downloaded)

In [67]:
# Download Files
pgn_paths: list[Path] = []
print(f"Downloading pgn files to {PGN_DATA_DIR}")
for i, pgn_url in enumerate(pgn_urls):
    file_name = Path(pgn_url).name
    file_path = PGN_DATA_DIR.joinpath(file_name)
    if file_path.exists():
        print(f"File {i+1} of {len(pgn_urls)} exists and will not be downloaded: {file_name}")
    
    else:
        print(f"\tDownloading file {i+1} of {len(pgn_urls)}: {file_name}")
        response = requests.get(pgn_url)
        file_path.write_bytes(response.content)

    pgn_paths.append(file_path)
print("Files downloaded")

File 1 of 4 exists and will not be downloaded: Pandora147s1_BronxNY_L2_rnvh3p1-8.txt
File 2 of 4 exists and will not be downloaded: Pandora180s1_BronxNY_L2_rnvh3p1-8.txt
File 3 of 4 exists and will not be downloaded: Pandora206s1_BuffaloNY_L2_rnvh3p1-8.txt
File 4 of 4 exists and will not be downloaded: Pandora55s1_QueensNY_L2_rnvh3p1-8.txt
Files downloaded


Build a datafram of all PGN data for columns with valid quality flags.

In [ ]:
# PGN format settings (these should not change)
pgn_section_delim = f"{'-'*87}\n"
header_delim = ": "
pgn_loc_key = "Short location name"
pgn_lat_key = "Location latitude [deg]"
pgn_lon_key = "Location longitude [deg]"
# Avogadro constant divided by 10000
no2_scale = 6.02214076E+19

# build the final dataset
pgn_data = pd.DataFrame()
print("PGN Loading Started")
for i, pgn_path in enumerate(pgn_paths):
    print(f"Loading file {i+1} of {len(pgn_paths)}: {pgn_path.name}")

    # get file format indicies
    file_suffix = pgn_path.name.split('_')[-1]
    file_data = pgn_formats.get(file_suffix)
    if file_data is None:
      raise Exception(f"Invalid suffix for {pgn_path}, handled suffixes:", pgn_formats)
    no2_quality_flag_index = file_data['no2_quality_flag_index']
    valid_quality_flags = file_data['valid_quality_flags']
    column_index = file_data['column_index']
    column_unc_index = file_data['column_unc_index']

    # get file sections as lines
    text = pgn_path.read_text()
    metadata_text, column_text, data_text = text.split(pgn_section_delim)
    metadata_lines = metadata_text.splitlines()
    column_lines = column_text.splitlines()
    data_lines = data_text.splitlines()

    # get metadata
    metadata = {}
    for line in metadata_lines:
        key, value = line.split(header_delim)
        metadata[key] = value

    # get data
    rows = []
    for line in data_lines:
      values = line.split()

      # ignore if timestamp is not between start and end time
      timestamp = datetime.fromisoformat(values[0]).replace(tzinfo=None)
      if not (start_date <= timestamp):
        continue

      # ignore row if quality is not between 0 and 10
      no2_quality_flag = int(values[no2_quality_flag_index])
      if no2_quality_flag not in valid_quality_flags:
        continue
      
      # Nitrogen dioxide tropospheric vertical column amount [moles per square meter]
      column = float(values[61])
      # Independent uncertainty of nitrogen dioxide tropospheric vertical column amount [moles per square meter]
      column_unc = float(values[62])

      lat_grid = float(metadata[pgn_lat_key])
      lon_grid = float(metadata[pgn_lon_key])
      loc = metadata[pgn_loc_key]
      row = {
         'Time': timestamp, 
         'Latitude': lat_grid, 
         'Longitude': lon_grid, 
         'Location': loc, 
         'Column': column*no2_scale, 
         'Uncertainty': column_unc*no2_scale
      }
      rows.append(row)
    df = pd.DataFrame(rows)
    if not rows:
      print("\tWARNING: No valid observations found (NO2 quality flag of 0 or 10)")
    else:
      print(f"\tValid Observations: {len(rows)}")
    pgn_data = pd.concat([pgn_data, df])
print(f"PGN Loading Complete, found {pgn_data.shape} valid observations.")
print("Writing to", PGN_DATA_PATH)
pgn_data.to_csv(PGN_DATA_PATH, index=False)
pgn_data.head()


PGN Loading Started
Loading file 1 of 4: Pandora147s1_BronxNY_L2_rnvh3p1-8.txt
Loading file 2 of 4: Pandora180s1_BronxNY_L2_rnvh3p1-8.txt
Loading file 3 of 4: Pandora206s1_BuffaloNY_L2_rnvh3p1-8.txt
	Valid Observations: 3202
Loading file 4 of 4: Pandora55s1_QueensNY_L2_rnvh3p1-8.txt
	Valid Observations: 10175
PGN Loading Complete, found (13377, 6) valid observations.
Writing to pgn-data\pgn-data.csv


,Timestamp,Latitude,Longitude,Location,Column,Uncertainty
0,2024-07-01 13:54:06.800,43.0015,-78.7869,BuffaloNY,7.549356e+14,1.135294e+14
1,2024-07-01 14:52:07.400,43.0015,-78.7869,BuffaloNY,7.286790e+14,1.090429e+14
2,2024-07-01 15:19:10.300,43.0015,-78.7869,BuffaloNY,4.645178e+14,1.056524e+14
3,2024-07-01 15:35:21.400,43.0015,-78.7869,BuffaloNY,6.737571e+14,1.039542e+14
4,2024-07-01 15:51:30.000,43.0015,-78.7869,BuffaloNY,4.410074e+14,1.003108e+14


Get latitudes and longitudes (for use with TEMPO download)

In [69]:
pgn_sites_df = pgn_data[['Location', 'Latitude', 'Longitude']].drop_duplicates().set_index('Location')
pgn_sites_df.head()

,Latitude,Longitude
Location,,
BuffaloNY,43.0015,-78.7869
QueensNY,40.7361,-73.8215


#### 1.1.1 Download TEMPO Data

Get TEMPO cloud links

In [70]:
short_name = 'TEMPO_NO2_L2' # collection name to search for in the EarthData
out_Q = 'NO2_trop_col_day'
version = 'V03'

cloud_files = []
for site, lat, lon in pgn_sites_df.itertuples():
    cloud_files += earthaccess.search_data(
        short_name = short_name,
        version = version,
        temporal = (start_date, end_date),
        point = (lon, lat)
    )

print("Found", len(cloud_files), "files.")

Found 160 files.


Get, and subset, TEMPO files (otherwise they are far too big)

In [ ]:
print("Starting TEMPO Download")
site_lats = pgn_sites_df.Latitude
site_lons = pgn_sites_df.Longitude

# how close a reading must be to a site to be considered
limit = 0.001

# this error comes from netCDF4 and is only a warning and can be ignored
warnings.filterwarnings(
    "ignore",
    message="__array__ implementation doesn't accept a copy keyword",
    category=DeprecationWarning,
)

print("Downloading files")
run_times = []
file_sizes = []
tempo_df = pd.read_csv(TEMPO_DATA_PATH) if TEMPO_DATA_PATH.exists() else pd.DataFrame()
for i, cloud_file in enumerate(cloud_files):
    print(f"Downloading file {i+1} of {len(cloud_files)}")
    try:
        start_time = datetime.now()

        # grab full file
        paths = earthaccess.download(cloud_file, local_path=TEMPO_DATA_DIR, show_progress=False)
        for path in paths:
            print("Parsing", path)

            # file name format: TEMPO_NO2_L2_V03_20240706T222907Z_S014G03.nc
            file_name = path.name
            name_parts = path.name.split('_')
            # time part format: 20240706T222907Z
            time_part = name_parts[4]
            file_time = datetime.strptime(time_part, "%Y%m%dT%H%M%%S")

            # extract geo data and NO2 data
            with nc.Dataset(path) as ds:
                geolocation = ds['geolocation']
                product =   ds['product']

                col_fil =   product['vertical_column_troposphere'].getncattr('_FillValue')
                unc_fil =   product['vertical_column_troposphere_uncertainty'].getncattr('_FillValue')

                times =     np.array(geolocation['time'])
                lat_grid =  np.array(geolocation['latitude'])
                lon_grid =  np.array(geolocation['longitude'])
                cols_grid = np.array(product['vertical_column_troposphere'])
                uncs_grid = np.array(product['vertical_column_troposphere_uncertainty'])

            # combine lat/lon pairs with times
            data = []
            for time, lats, lons, cols, uncs in zip(times, lat_grid, lon_grid, cols_grid, uncs_grid):
                for lat, lon, col, unc in zip(lats, lons, cols, uncs):
                    data.append([time, lat, lon, col, unc])
            
            # convert data into a dataframe
            file_df = pd.DataFrame(
                data, 
                columns=['Time', 'Latitude', 'Longitude', 'Column', 'Uncertainty']
            )

            # discard points that are not near a site
            all_site_mask = pd.Series(False, index=file_df.index)
            for lat, lon in zip(site_lats, site_lons):
                lat_mask = (file_df.Latitude - lat).abs() <= limit
                lon_mask = (file_df.Longitude - lon).abs() <= limit
                site_mask = lat_mask | lon_mask
                all_site_mask = all_site_mask | site_mask
            
            # clean the data (replace fill values and drop na's)
            file_df = file_df.replace(col_fil, np.nan).replace(unc_fil, np.nan).dropna()
            tempo_df = pd.concat([tempo_df, file_df], ignore_index=True)

            # remove the .nc file and recreate the .nc file as an empty stub to keep it form being downloaded agian
            path.unlink()
            path.with_suffix('.nc').touch()
        
        # print performance
        files_remaining = len(cloud_files) - (i+1)

        # estimate run time
        run_time =(datetime.now() - start_time).total_seconds()
        # ignore short run times (they happen when something is skipped)
        if run_time > 2:
            run_times.append(run_time)
            avg_run_time = sum(run_times)/len(run_times)
            seconds_remaining = avg_run_time * files_remaining
            time_remaining = timedelta(seconds=seconds_remaining)
        
            print("Time Remaining:", time_remaining)
        
    except Exception as e:
        print("Skipping file:  Error occured while processing", cloud_file, "Error:", e)

# load data
tempo_df.to_csv(TEMPO_DATA_PATH, index=False)
    
print("File Download Complete")

Starting TEMPO Download
Parsing tempo-data\TEMPO_NO2_L2_V03_20240701T104719Z_S001G03.nc
Skipping file:  Error occured while processing Collection: {'ShortName': 'TEMPO_NO2_L2', 'Version': 'V03'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Latitude': 57.99621582, 'Longitude': -76.845062256}, {'Latitude': 54.233428955, 'Longitude': -78.294830322}, {'Latitude': 50.046085358, 'Longitude': -79.55393219}, {'Latitude': 45.393371582, 'Longitude': -80.641090393}, {'Latitude': 40.244720459, 'Longitude': -81.57183075}, {'Latitude': 35.1705513, 'Longitude': -82.285873413}, {'Latitude': 29.197639465, 'Longitude': -82.933853149}, {'Latitude': 22.799566269, 'Longitude': -83.459037781}, {'Latitude': 17.253734589, 'Longitude': -83.781524658}, {'Latitude': 17.301038742, 'Longitude': -77.914489746}, {'Latitude': 23.61391449, 'Longitude': -77.201324463}, {'Latitude': 29.539440155, 'Longitude': -76.292076111}, {'Latitude': 35.069496155, 'Longitude': 

### 1.2. Explore Data

#### 1.2.1 Explore Pandora

Load Pandora Data

In [111]:
pgn_data = pd.read_csv(PGN_DATA_PATH)
# convert datetime to seconds since the epoch (to match tempo)
nanoseconds_since_epoch = pd.to_datetime(pgn_data.Timestamp, utc=True)
seconds_since_epoch = nanoseconds_since_epoch.astype('int64') // 10**9
pgn_data['Timestamp'] = seconds_since_epoch
pgn_data = pgn_data.rename(columns={'Timestamp': 'Time'})
pgn_data.head()

,Time,Latitude,Longitude,Location,Column,Uncertainty
0,1719842046,43.0015,-78.7869,BuffaloNY,7.549356e+14,1.135294e+14
1,1719845527,43.0015,-78.7869,BuffaloNY,7.286790e+14,1.090429e+14
2,1719847150,43.0015,-78.7869,BuffaloNY,4.645178e+14,1.056524e+14
3,1719848121,43.0015,-78.7869,BuffaloNY,6.737571e+14,1.039542e+14
4,1719849090,43.0015,-78.7869,BuffaloNY,4.410074e+14,1.003108e+14


Get Pandora Stats

In [ ]:
pgn_data[['Column', 'Uncertainty']].describe()

,Column,Uncertainty
count,1.337700e+04,1.337700e+04
mean,6.289382e+15,9.779774e+13
std,5.370501e+15,2.841439e+13
min,-1.635613e+16,3.743784e+13
25%,2.565372e+15,8.028718e+13
50%,4.886726e+15,9.432479e+13
75%,8.262979e+15,1.087478e+14
max,5.084433e+16,4.669327e+14


#### Explore TEMPO

Load TEMPO

In [122]:
tempo_data = pd.read_csv(TEMPO_DATA_PATH).rename(columns={
    'vertical_column_troposphere': 'Column',
    'vertical_column_troposphere_uncertainty': "Uncertainty"
})
tempo_data.head()

,Time,Latitude,Longitude,Column,Uncertainty
0,1.403866e+09,58.582800,-64.220375,5.018798e+14,1.366402e+14
1,1.403866e+09,58.536476,-64.263855,6.264613e+14,2.370017e+14
2,1.403866e+09,58.496826,-64.299690,6.328754e+14,2.307473e+14
3,1.403866e+09,58.458267,-64.334200,3.225646e+14,3.939703e+14
4,1.403866e+09,58.421070,-64.367096,2.493699e+14,2.303265e+14


In [123]:
tempo_data[['Column', 'Uncertainty']].describe()

,Column,Uncertainty
count,4.472012e+06,4.472012e+06
mean,1.257430e+15,1.291481e+15
std,2.213436e+15,1.691506e+15
min,-1.166274e+17,1.066555e+14
25%,2.362738e+14,5.334135e+14
50%,8.152280e+14,7.879533e+14
75%,1.723481e+15,1.346081e+15
max,2.927148e+17,2.446642e+17


Compare datasets
Ensure datasets are similar.

In [124]:
raw_compare_df = pgn_data[['Column', 'Uncertainty']].describe().join(
    tempo_data[['Column', 'Uncertainty']].describe(),
    lsuffix='_PGN',
    rsuffix='_TEMPO'
)
raw_compare_df

,Column_PGN,Uncertainty_PGN,Column_TEMPO,Uncertainty_TEMPO
count,1.337700e+04,1.337700e+04,4.472012e+06,4.472012e+06
mean,6.289382e+15,9.779774e+13,1.257430e+15,1.291481e+15
std,5.370501e+15,2.841439e+13,2.213436e+15,1.691506e+15
min,-1.635613e+16,3.743784e+13,-1.166274e+17,1.066555e+14
25%,2.565372e+15,8.028718e+13,2.362738e+14,5.334135e+14
50%,4.886726e+15,9.432479e+13,8.152280e+14,7.879533e+14
75%,8.262979e+15,1.087478e+14,1.723481e+15,1.346081e+15
max,5.084433e+16,4.669327e+14,2.927148e+17,2.446642e+17


We can see that the mean and standard deviation of Column values are within the an order of magnitude but uncertainty is not.  They differ some but they are expected to vary due to the fact that TEMPO only collects data for part of the day.  For now, I will only use Column and investigate Uncertainty later.

**Future Work**<br>
Investigate the differences in values between the two datasets.


### 1.3. Clean Data

#### 1.3.1 Clean Pandora 

**Future Work**<br>
For now I only use raw data with outliers removed.  Consider performing the following steps to further normalize datasets.
1) Seasonal Composition
1) Standardized Data

In [77]:
def get_outlier_mask(column: pd.Series) -> pd.Series:
    """
    Returns a mask where outliers are False for use in filtering outliers.
    Outliers are defined as more than 3 standard deviations from the mean.
    Intended for use in filtering out outliers.
    use: 
        mask = get_outlier_mask(df.ColumnOfIntrest)
        df = df[mask]

    ARGS:
        column (pd.Series[float]): the column to build a mask for
    
    RETURN
        pd.Series[bool]: A mask of the outliers
    """
    col_std = column.std()
    col_mean = column.mean()
    min_cutoff = col_mean - (3 * col_std)
    max_cutoff = col_mean + (3 * col_std)
    return (min_cutoff < column) & (column < max_cutoff)

Raw Data Cleaning

In [125]:
col_mask = get_outlier_mask(pgn_data.Column)
unc_mask = get_outlier_mask(pgn_data.Uncertainty)
mask = col_mask & unc_mask

print("Filtering", sum(~mask), "rows of", len(mask), f"due to outliers, {round(sum(~mask)/len(mask)*100)}%")
pgn_clean_data = pgn_data[mask]
pgn_clean_data.head()

Filtering 346 rows of 13377 due to outliers, 3%


,Time,Latitude,Longitude,Location,Column,Uncertainty
0,1719842046,43.0015,-78.7869,BuffaloNY,7.549356e+14,1.135294e+14
1,1719845527,43.0015,-78.7869,BuffaloNY,7.286790e+14,1.090429e+14
2,1719847150,43.0015,-78.7869,BuffaloNY,4.645178e+14,1.056524e+14
3,1719848121,43.0015,-78.7869,BuffaloNY,6.737571e+14,1.039542e+14
4,1719849090,43.0015,-78.7869,BuffaloNY,4.410074e+14,1.003108e+14


#### 1.3.2 Clean TEMPO

### 1.4. Transform Data

In [128]:
col_mask = get_outlier_mask(tempo_data.Column)
unc_mask = get_outlier_mask(tempo_data.Uncertainty)
mask = col_mask & unc_mask

print("Filtering", sum(~mask), "rows of", len(mask), f"due to outliers, {round(sum(~mask)/len(mask)*100)}%")
tempo_clean_data = tempo_data[mask]
tempo_clean_data.head()

Filtering 122548 rows of 4472012 due to outliers, 3%


,Time,Latitude,Longitude,Column,Uncertainty
0,1.403866e+09,58.582800,-64.220375,5.018798e+14,1.366402e+14
1,1.403866e+09,58.536476,-64.263855,6.264613e+14,2.370017e+14
2,1.403866e+09,58.496826,-64.299690,6.328754e+14,2.307473e+14
3,1.403866e+09,58.458267,-64.334200,3.225646e+14,3.939703e+14
4,1.403866e+09,58.421070,-64.367096,2.493699e+14,2.303265e+14


Compare clean data

In [129]:
clean_compare_df = pgn_clean_data[['Column', 'Uncertainty']].describe().join(
    tempo_clean_data[['Column', 'Uncertainty']].describe(),
    lsuffix='_PGN',
    rsuffix='_TEMPO'
)
clean_compare_df

,Column_PGN,Uncertainty_PGN,Column_TEMPO,Uncertainty_TEMPO
count,1.303100e+04,1.303100e+04,4.349464e+06,4.349464e+06
mean,5.773519e+15,9.517311e+13,1.092413e+15,1.106714e+15
std,4.212874e+15,2.234539e+13,1.393054e+15,9.597021e+14
min,-9.081990e+15,3.743784e+13,-5.380277e+15,1.066555e+14
25%,2.513069e+15,7.985961e+13,2.308500e+14,5.281086e+14
50%,4.743219e+15,9.366838e+13,7.939537e+14,7.705515e+14
75%,7.877863e+15,1.073386e+14,1.648714e+15,1.267311e+15
max,2.237526e+16,1.823444e+14,7.897732e+15,6.365967e+15


Git differences between raw and clean data comparison

In [130]:
raw_compare_df

,Column_PGN,Uncertainty_PGN,Column_TEMPO,Uncertainty_TEMPO
count,1.337700e+04,1.337700e+04,4.472012e+06,4.472012e+06
mean,6.289382e+15,9.779774e+13,1.257430e+15,1.291481e+15
std,5.370501e+15,2.841439e+13,2.213436e+15,1.691506e+15
min,-1.635613e+16,3.743784e+13,-1.166274e+17,1.066555e+14
25%,2.565372e+15,8.028718e+13,2.362738e+14,5.334135e+14
50%,4.886726e+15,9.432479e+13,8.152280e+14,7.879533e+14
75%,8.262979e+15,1.087478e+14,1.723481e+15,1.346081e+15
max,5.084433e+16,4.669327e+14,2.927148e+17,2.446642e+17


In [131]:
clean_compare_df

,Column_PGN,Uncertainty_PGN,Column_TEMPO,Uncertainty_TEMPO
count,1.303100e+04,1.303100e+04,4.349464e+06,4.349464e+06
mean,5.773519e+15,9.517311e+13,1.092413e+15,1.106714e+15
std,4.212874e+15,2.234539e+13,1.393054e+15,9.597021e+14
min,-9.081990e+15,3.743784e+13,-5.380277e+15,1.066555e+14
25%,2.513069e+15,7.985961e+13,2.308500e+14,5.281086e+14
50%,4.743219e+15,9.366838e+13,7.939537e+14,7.705515e+14
75%,7.877863e+15,1.073386e+14,1.648714e+15,1.267311e+15
max,2.237526e+16,1.823444e+14,7.897732e+15,6.365967e+15


Filtering did not change the disparity between Uncertainty, it cannot be considered for now.  TEMPO numbers seem to be consistantly a bit lower, but this may be due to the fact that TEMPO data is only collected durrind daylight hours.  For now I will use both to build a model.

**Future Work**
Investigate difference in PGN and TEMPO readings.

## 2. Data Modeling

### 2.1 Data Prep

In [132]:
all_data = pd.concat([pgn_clean_data, tempo_clean_data])
all_data.head()

,Time,Latitude,Longitude,Location,Column,Uncertainty
0,1.719842e+09,43.0015,-78.7869,BuffaloNY,7.549356e+14,1.135294e+14
1,1.719846e+09,43.0015,-78.7869,BuffaloNY,7.286790e+14,1.090429e+14
2,1.719847e+09,43.0015,-78.7869,BuffaloNY,4.645178e+14,1.056524e+14
3,1.719848e+09,43.0015,-78.7869,BuffaloNY,6.737571e+14,1.039542e+14
4,1.719849e+09,43.0015,-78.7869,BuffaloNY,4.410074e+14,1.003108e+14


In [133]:
all_data.describe()

,Time,Latitude,Longitude,Column,Uncertainty
count,4.362495e+06,4.362495e+06,4.362495e+06,4.362495e+06,4.362495e+06
mean,1.404901e+09,3.533997e+01,-7.798717e+01,1.106396e+15,1.103693e+15
std,1.831755e+07,1.135126e+01,3.389115e+00,1.432855e+15,9.598571e+14
min,1.403866e+09,1.733732e+01,-8.383506e+01,-9.081990e+15,3.743784e+13
25%,1.403876e+09,2.550596e+01,-8.045638e+01,2.325918e+14,5.263882e+14
50%,1.403890e+09,3.436251e+01,-7.847244e+01,7.975274e+14,7.687020e+14
75%,1.403908e+09,4.451519e+01,-7.622627e+01,1.658236e+15,1.264721e+15
max,1.760807e+09,5.863553e+01,-6.396864e+01,2.237526e+16,6.365967e+15


Convert time to Time of Day

In [155]:
seconds_per_day = 86400
seconds_since_start_of_day = all_data.Time % seconds_per_day
all_data['TimeOfDay'] = seconds_since_start_of_day.astype(int)

For simplicity (and due to a file format issue that can be fixed later), times will be ignored.

**Future Work**<br>
Use time of day as a feature

Split Data

In [178]:
from sklearn.model_selection import train_test_split

X_cols = ['Time', 'Latitude', 'Longitude']
X = all_data[X_cols]
y = all_data['Column']

X_train, X_test, y_train, y_test = train_test_split(X, y)

### 2.2 Train Models

For proof of concept, lets try a 2nd degree polynomial regression.<br>
This seems like a intitive fit as NO2 buidls durring the day then dies off overnight.

In [179]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


# create the kernal transformation
poly = PolynomialFeatures(degree=2)

# apply the kernal transformation to make data linear
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# train a linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


### 2.2 Test Models

In [180]:
from sklearn.metrics import mean_squared_error


y_pred = model.predict(X_test_poly)
mse = mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error (MSE): {np.sqrt(mse):.2e}")

Root Mean Squared Error (MSE): 1.39e+15


### 2.3. Model Visualization

In [ ]:
from matplotlib import pyplot as plt


lat = 40.7361
lon = -73.8215
times = list(range(86400))

X = pd.DataFrame([[time, lat, lon] for time in times], columns=['Time', 'Latitude', 'Longitude'])
y = model.predict(X)


    


Comparing to Site: (np.float64(40.7361), np.float64(-73.8215))


c:\Users\natha\Documents\Classes\MST\MLA\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


ValueError: X has 3 features, but LinearRegression is expecting 10 features as input.